# Proyecto Final del curso de Ingeniería de Datos 

Se propone crear un pipeline que extraiga datos de una API pública de forma constante combinándolos con información extraída de una base de datos y colocándolos en un Data Warehouse.

## Setup

### Instalación de librerias

In [1]:
# Instalacion de la libreria para interactuar con la base de datos, especificamente con Postgres
#%pip install sqlalchemy psycopg2-binary

### Importación de librerias

In [106]:
# Libreria para interactuar con APIs
import requests

import pandas as pd

# Libreria para interactuar con la base de datos
import sqlalchemy as sa
from configparser import ConfigParser
from pathlib import Path

import psycopg2

### Definición de funciones

In [66]:
def read_api_credentials(config_file: Path, section: str) -> dict:
    """
    Lee las credenciales de la API desdde un archivo de configuracion

    Parametros:
    config_file: Ruta del archivo de configuracion
    section: seccion del archivo de configuracion que contiene las credenciales
    """
    config = ConfigParser()
    config.read(config_file)
    api_credentials = dict(config[section])
    return api_credentials

## Conexion con la API

Extraccion de datos de la API de transporte de Buenos Aires

In [67]:
base_url = "https://apitransporte.buenosaires.gob.ar"

api_keys = read_api_credentials("config/pipeline.conf", "api_transporte")

# No pude con los headers, lo puse como parametros pero oculte la info
params = { 
    "client_id" : api_keys["client_id"],
    "client_secret" : api_keys["client_secret"]
}

In [78]:
# Parametro que a veces es requisitos
formato_json = {'json': 1}

### Extracción de datos de los bus

In [68]:
endpoint_bus = "colectivos"

_____________

Para buses en especifico, si quiero el general no corro ninguna de estas lineas

In [69]:
# LA NUEVA METROPOL S.A.
la_nueva_metropol = {'agency_id': 9}

In [70]:
# MICRO OMNIBUS PRIMERA JUNTA S.A
primera_junta = {'agency_id': 145}

In [ ]:
# TRANSPORTE AUTOMOTORES LA PLATA SA
talp = {'agency_id': 155}

______________

#### Información de la posicion de los bus

In [100]:
# Obtencion de la posición de los vehículos monitoreados actualizada cada 30 segundos. 
# Si no se pasan parámetros de entrada, retorna la posición actual de todos los vehículos monitoreados.

endpoint_busPositions = f"{endpoint_bus}/vehiclePositionsSimple"

full_url_busPositions = f"{base_url}/{endpoint_busPositions}"

params_busPositions = params.copy()

In [101]:
params_busPositions.update(primera_junta)

In [102]:
r_busPositions = requests.get(full_url_busPositions, params=params_busPositions)

r_busPositions.status_code

200

In [73]:
json_busData = r_busPositions.json()
json_busData

[{'route_id': '1279',
  'latitude': -34.81734,
  'longitude': -58.19756,
  'speed': 6.944444,
  'timestamp': 1707247498,
  'id': '23691',
  'direction': 0,
  'agency_name': 'MICRO OMNIBUS PRIMERA JUNTA S.A',
  'agency_id': 145,
  'route_short_name': '324R3',
  'tip_id': '82341-1',
  'trip_headsign': 'A - Barrio Sitra - IDA'},
 {'route_id': '1289',
  'latitude': -34.70458,
  'longitude': -58.288105,
  'speed': 5.833333,
  'timestamp': 1707247500,
  'id': '23697',
  'direction': 0,
  'agency_name': 'MICRO OMNIBUS PRIMERA JUNTA S.A',
  'agency_id': 145,
  'route_short_name': '324R6P',
  'tip_id': '82933-1',
  'trip_headsign': 'Ramal B - a A. Bello'},
 {'route_id': '1284',
  'latitude': -34.7874,
  'longitude': -58.2557,
  'speed': 10,
  'timestamp': 1707247500,
  'id': '23726',
  'direction': 1,
  'agency_name': 'MICRO OMNIBUS PRIMERA JUNTA S.A',
  'agency_id': 145,
  'route_short_name': '324R3F',
  'tip_id': '82579-1',
  'trip_headsign': 'B - Barrio Centenario (por Milan) - VUELTA'},
 {'

In [74]:
type(json_busData)

list

In [75]:
json_busData[1].keys()

dict_keys(['route_id', 'latitude', 'longitude', 'speed', 'timestamp', 'id', 'direction', 'agency_name', 'agency_id', 'route_short_name', 'tip_id', 'trip_headsign'])

Para pasar a un dataframe la data


In [76]:
# Para pasar el json a una dataframe
df_busPositions = pd.json_normalize(json_busData)
df_busPositions.sample(n=10)

,route_id,latitude,longitude,speed,timestamp,id,direction,agency_name,agency_id,route_short_name,tip_id,trip_headsign
45,1286,-34.73891,-58.273160,2.500000,1707247500,23966,1,MICRO OMNIBUS PRIMERA JUNTA S.A,145,324T5,82799-1,B - Barrio Centenario (por Milan) - VUELTA
18,1295,-34.98296,-58.148130,19.444445,1707247500,23829,0,MICRO OMNIBUS PRIMERA JUNTA S.A,145,324R9F,83232-1,a Pilar x Ford
65,1293,-34.76808,-58.272260,0.000000,1707247498,25702,0,MICRO OMNIBUS PRIMERA JUNTA S.A,145,324R9,83126-1,a Moreno x Panamericana
21,1298,-34.74244,-58.260240,3.888888,1707247500,23854,1,MICRO OMNIBUS PRIMERA JUNTA S.A,145,324R16,83444-1,a Pte. Saavedra
33,1297,-34.73371,-58.263140,0.000000,1707247500,23918,0,MICRO OMNIBUS PRIMERA JUNTA S.A,145,324R16,83362-1,Ramal 13 - IDA
47,1291,-34.70769,-58.275370,9.444444,1707247500,23972,0,MICRO OMNIBUS PRIMERA JUNTA S.A,145,324R6C,83030-1,a Moreno x Virreyes
1,1289,-34.70458,-58.288105,5.833333,1707247500,23697,0,MICRO OMNIBUS PRIMERA JUNTA S.A,145,324R6P,82933-1,Ramal B - a A. Bello
16,1297,-34.72504,-58.261880,0.000000,1707247500,23811,0,MICRO OMNIBUS PRIMERA JUNTA S.A,145,324R16,83359-1,Ramal 13 - IDA
42,1294,-34.72806,-58.265180,10.555555,1707247500,23959,1,MICRO OMNIBUS PRIMERA JUNTA S.A,145,324R9,83175-1,a Pte. Saavedra
60,1296,-34.74689,-58.264840,5.000000,1707247498,24172,1,MICRO OMNIBUS PRIMERA JUNTA S.A,145,324R9F,83291-1,a Pte. Saavedra


#### Informacion de servicios de alerta

In [85]:
# Las alertas de servicio te permiten proporcionar actualizaciones cada vez que se produce una interrupción en la red. 
# Las demoras y cancelaciones de viajes individuales a menudo se deben comunicar a través de los trip updates.

endpoint_busServiceAlerts = f"{endpoint_bus}/serviceAlerts"

full_url_busServiceAlerts = f"{base_url}/{endpoint_busServiceAlerts}"

params_busServiceAlerts = params.copy()
params_busServiceAlerts.update(formato_json)


In [86]:
r_busServiceAlerts = requests.get(full_url_busServiceAlerts, params=params_busServiceAlerts)

r_busServiceAlerts.status_code

200

In [87]:
json_busData = r_busServiceAlerts.json()
json_busData

{'_entity': [{'_alert': {'_active_period': [],
    '_cause': 2,
    '_description_text': {'_translation': [{'_language': '',
       '_text': 'Por desvío, la linea VERDE se detiene acá',
       'extensionObject': None}],
     'extensionObject': None},
    '_effect': 4,
    '_header_text': {'_translation': [{'_language': '',
       '_text': 'Paradas provisorias de la línea VERDE',
       'extensionObject': None}],
     'extensionObject': None},
    '_informed_entity': [{'_agency_id': '',
      '_route_id': None,
      '_route_type': 0,
      '_stop_id': '73001',
      '_trip': None,
      'extensionObject': None},
     {'_agency_id': '',
      '_route_id': None,
      '_route_type': 0,
      '_stop_id': '15011',
      '_trip': None,
      'extensionObject': None}],
    '_url': {'_translation': [], 'extensionObject': None},
    'extensionObject': None},
   '_id': '5274244',
   '_is_deleted': False,
   '_trip_update': None,
   '_vehicle': None,
   'extensionObject': None},
  {'_alert': {'_

In [88]:
# Para 323 Micro Omnibus primera junta
params_busServiceAlerts.update(primera_junta)

r_busServiceAlerts = requests.get(full_url_busServiceAlerts, params=params_busServiceAlerts)

r_busServiceAlerts.status_code

200

In [89]:
json_busData = r_busServiceAlerts.json()
json_busData

{'_entity': [],
 '_header': {'_gtfs_realtime_version': '1.0',
  '_incrementality': 0,
  '_timestamp': 1707248574,
  'extensionObject': None},
 'extensionObject': None}

In [90]:
type(json_busData)

dict

In [91]:
json_busData.keys()

dict_keys(['_entity', '_header', 'extensionObject'])

In [93]:
data_busSA= json_busData['_entity']
df_busSA = pd.DataFrame(data_busSA)
df_busSA

""


### Extracción de datos del estado de las estaciones de las ecobicis

In [47]:
# Obtencion del número de bicicletas y anclajes disponibles en cada estación y disponibilidad de estación.

endpoint_ecobiciSS = "ecobici/gbfs/stationStatus"
full_url_ecobiciSS = f"{base_url}/{endpoint_ecobiciSS}"

r_ecobiciSS = requests.get(full_url_ecobiciSS, params=params)

In [48]:
r_ecobiciSS.status_code

200

In [49]:
json_ecobiciSS = r_ecobiciSS.json()
json_ecobiciSS

{'last_updated': 1707186551,
 'ttl': 8,
 'data': {'stations': [{'station_id': '2',
    'num_bikes_available': 13,
    'num_bikes_available_types': {'mechanical': 13, 'ebike': 0},
    'num_bikes_disabled': 2,
    'num_docks_available': 25,
    'num_docks_disabled': 0,
    'last_reported': 1707186544,
    'is_charging_station': False,
    'status': 'IN_SERVICE',
    'is_installed': 1,
    'is_renting': 1,
    'is_returning': 1,
    'traffic': None},
   {'station_id': '3',
    'num_bikes_available': 5,
    'num_bikes_available_types': {'mechanical': 5, 'ebike': 0},
    'num_bikes_disabled': 2,
    'num_docks_available': 21,
    'num_docks_disabled': 0,
    'last_reported': 1707186340,
    'is_charging_station': False,
    'status': 'IN_SERVICE',
    'is_installed': 1,
    'is_renting': 1,
    'is_returning': 1,
    'traffic': None},
   {'station_id': '4',
    'num_bikes_available': 3,
    'num_bikes_available_types': {'mechanical': 3, 'ebike': 0},
    'num_bikes_disabled': 0,
    'num_doc

In [50]:
json_ecobiciSS.keys()

dict_keys(['last_updated', 'ttl', 'data'])

In [51]:
# Para pasar el json a una dataframe

data_ecobiciSS= json_ecobiciSS['data']['stations']
df_ecobiciSS = pd.DataFrame(data_ecobiciSS)

df_ecobiciSS.sample(n=10)

,station_id,num_bikes_available,num_bikes_available_types,num_bikes_disabled,num_docks_available,num_docks_disabled,last_reported,is_charging_station,status,is_installed,is_renting,is_returning,traffic
284,434,1,"{'mechanical': 1, 'ebike': 0}",0,19,0,1.707186e+09,False,IN_SERVICE,1,1,1,None
256,385,7,"{'mechanical': 7, 'ebike': 0}",0,9,0,1.707187e+09,False,IN_SERVICE,1,1,1,None
290,446,0,"{'mechanical': 0, 'ebike': 0}",0,0,0,NaN,False,END_OF_LIFE,1,0,0,None
45,65,3,"{'mechanical': 3, 'ebike': 0}",1,16,0,1.707186e+09,False,IN_SERVICE,1,1,1,None
124,181,1,"{'mechanical': 1, 'ebike': 0}",4,15,0,1.707186e+09,False,IN_SERVICE,1,1,1,None
82,117,4,"{'mechanical': 4, 'ebike': 0}",2,14,0,1.707186e+09,False,IN_SERVICE,1,1,1,None
81,116,1,"{'mechanical': 1, 'ebike': 0}",1,10,0,1.707186e+09,False,IN_SERVICE,1,1,1,None
79,112,1,"{'mechanical': 1, 'ebike': 0}",2,17,0,1.707186e+09,False,IN_SERVICE,1,1,1,None
207,311,8,"{'mechanical': 8, 'ebike': 0}",0,12,0,1.707186e+09,False,IN_SERVICE,1,1,1,None
148,213,1,"{'mechanical': 1, 'ebike': 0}",2,17,0,1.707186e+09,False,IN_SERVICE,1,1,1,None


## Conexión con base de datos

In [115]:
db_keys = read_api_credentials("config/pipeline.conf", "RedShift")

try:
    conn = psycopg2.connect(
        host = db_keys["host"],
        dbname = db_keys["dbname"] ,
        user = db_keys['user'],
        password = db_keys['pwd'],
        port = db_keys["port"],
    )
    print("Conectado a Redshift con éxito!")
    
except Exception as e:
    print("No es posible conectar a Redshift")
    print(e)

Conectado a Redshift con éxito!


### Tablas para datos de los bus

Tabla para las agencias de interes

In [109]:
with conn.cursor() as cur:
    cur.execute("""
        create table if not exists  camilagonzalezalejo02_coderhouse.agencies
        (       	
	    agency_id INTEGER,
	    agency_name VARCHAR(100)
        )
    sortkey(agency_id)
    """)
    conn.commit()

Tabla para los viajes realizados por dichas agencias

In [116]:
try:
    with conn.cursor() as cur:
        cur.execute("""
            create table if not exists  camilagonzalezalejo02_coderhouse.positions
            (	
            id INTEGER,
            agency_id INTEGER,
            route_id INTEGER,
            latitude NUMERIC,
            longitude NUMERIC,
            speed NUMERIC,
            route_short_name VARCHAR(50),
            trip_headsign VARCHAR(100),
            date_and_time timestamp 
            )   
        """)
        conn.commit()
except psycopg2.Error as e:
    print("Error al ejecutar la consulta SQL:", e)